# Procédure de la récupération des données Météo France SWI 1969—2022, mailles et métadonnées

## I. Télécharger les données

1. Se rendre à la [page dédiée][1]:
   * télécharger l'archive zip avec toutes les mailles depuis la rubrique `Téléchargement` ;
   * télécharger le fichier `Coordonnées des mailles` depuis la rubrique `Documentation`.

[1]: https://donneespubliques.meteofrance.fr/?fond=produit&id_produit=301&id_rubrique=40

### Les données du fichier `metadonnees_swi_276.csv`

Légende :
* `num_maille` : numéro de la maille
* `lambx`, `lamby` : coordonnées de la maille en Lambert 2 étendu, en hectomètres
* `lambx93`, `lamby93` : coordonnées de la maille en Lambert 93
* `lat_dg`, `lon_dg` : coordonnées de la maille en degrés décimaux

In [16]:
import pandas as pd

df_geo_coords = pd.read_csv('metadonnees_swi_276.csv', sep=';')
df_geo_coords

,num_maille,lambx,lamby,lambx93,lamby93,lat_dg,lon_dg
0,2,5880,26730,641374,7106309,51.0497,2.16654
1,3,5960,26730,649370,7106242,51.0498,2.28031
2,4,6040,26730,657366,7106175,51.0498,2.39409
3,5,6120,26730,665362,7106108,51.0497,2.50786
4,6,6200,26730,673358,7106041,51.0495,2.62163
...,...,...,...,...,...,...,...
8976,9888,11800,16330,1223902,6062151,41.4739,9.26140
8977,9889,11640,16250,1207852,6054302,41.4148,9.06318
8978,9890,11720,16250,1215842,6054231,41.4087,9.15811
8979,9891,11800,16250,1223832,6054161,41.4024,9.25302


## II. Attribuer les coordonnées géographiques (fichier `metadonnees_swi_276.csv`) à chacun des fichiers contenant les données

1. Parcourir les fichiers CSV contenant les données et les lire en tant que DataFrame. Veiller à indiquer le point-virgule en tant que séparateur et la virgule en tant que séparateur décimal.
2. Attribuer les coordonnées géographiques en effectuant la jointure avec le fichier `metadonnees_swi_276.csv`.
3. Supprimer toutes les colonnes sauf la latitude, la longitude, l'index SWI et la date.
4. Renommer les colonnes en minuscule.
5. Sauvegarder les fichier CSV ainsi modifiés.

In [33]:
import os
import pandas as pd
from tqdm import tqdm

INPUT_DATA_DIR = 'SWI_Package_1969-2022'
OUTPUT_DATA_DIR = 'output'
# if output directory doesn't exist, create it:
if not os.path.exists(OUTPUT_DATA_DIR):
    os.makedirs(OUTPUT_DATA_DIR)
# remove all files from the output dir:
for filename in os.listdir(OUTPUT_DATA_DIR):
    os.remove(f'{OUTPUT_DATA_DIR}/{filename}')

# loop over files in the folder SWI_Package_1969-2022
for filename in tqdm(os.listdir(INPUT_DATA_DIR)):
    path_to_csv = f'{INPUT_DATA_DIR}/{filename}'
    df_without_geo_coords = pd.read_csv(path_to_csv, sep=';', decimal=',')
    df_with_geo_coords = pd.merge(df_without_geo_coords, df_geo_coords, how='left', left_on=['NUMERO', 'LAMBX', 'LAMBY'], right_on=['num_maille', 'lambx93', 'lamby93'])
    df_with_geo_coords.drop(['NUMERO', 'LAMBX', 'LAMBY', 'num_maille', 'lambx', 'lamby', 'lambx93', 'lamby93'], axis=1, inplace=True)
    df_with_geo_coords.rename(columns={'lat_dg': 'latitude_y', 
                                       'lon_dg': 'longitude_x',
                                       'SWI_UNIF_MENS3': 'soil_wetness_index_monthly_avg',
                                       'DATE': 'date'}, inplace=True)
    df_with_geo_coords['date'] = pd.to_datetime(df_with_geo_coords['date'], format='%Y%m')
    df_with_geo_coords.to_csv(f'{OUTPUT_DATA_DIR}/{filename}', index=False)

100%|██████████| 40/40 [00:50<00:00,  1.27s/it]
